In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
import os.path as osp
from tqdm import tqdm
def calculate_high_frequency_energy(image, threshold_radius=50):
    # Step 1: Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Step 2: Apply FFT and shift the zero frequency to the center
    f = np.fft.fft2(gray_image)
    fshift = np.fft.fftshift(f)
    
    # Step 3: Get the magnitude spectrum
    magnitude_spectrum = np.abs(fshift)
    
    # Step 4: Create a mask to isolate high-frequency components
    rows, cols = gray_image.shape
    crow, ccol = rows // 2 , cols // 2  # Center of the frequency image
    mask = np.ones((rows, cols), np.uint8)
    cv2.circle(mask, (ccol, crow), threshold_radius, 0, thickness=-1)  # Remove low-frequency components
    
    # Step 5: Apply the mask
    high_freq_magnitude = magnitude_spectrum * mask
    
    # Step 6: Calculate energy (sum of squared magnitudes in high-frequency region)
    high_frequency_energy = np.sum(np.square(high_freq_magnitude))
    
    return high_frequency_energy, magnitude_spectrum, mask, high_freq_magnitude



In [16]:
def high_frequency_energy(path):
    energies = []
    for path in tqdm(glob(osp.join(path, "images_4", "*"))):
        image = cv2.imread(path)
        high_frequency_energy, magnitude_spectrum, mask, high_freq_magnitude = calculate_high_frequency_energy(image)
        energies.append(np.log(high_frequency_energy))
    energies = np.array(energies)
    return energies.mean()

In [17]:
for scene in ["bicycle", "bonsai", "counter", "flowers", "garden", "kitchen", "room", "stump", "treehill"]:
    root = f"data/mipnerf360/{scene}"
    print(f"{scene} : {high_frequency_energy(root)}")

100%|██████████| 194/194 [00:21<00:00,  9.22it/s]


bicycle : 34.02963987620036


100%|██████████| 584/584 [00:15<00:00, 38.67it/s]


bonsai : 30.900883839071852


100%|██████████| 240/240 [00:08<00:00, 29.70it/s]


...

room : 30.673900297309288


100%|██████████| 125/125 [00:08<00:00, 14.56it/s]


stump : 33.48920988198713


100%|██████████| 141/141 [00:12<00:00, 10.92it/s]

treehill : 34.0563395085374
